In [2]:
import pandas as pd
import numpy as np
import sys
import os

module_path = os.path.join(os.getcwd(), '..', 'features')
sys.path.append(module_path)
from utils import preprocess_sales

import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import (
    ARIMA, AutoARIMA, 
    ExponentialSmoothing,
    Prophet,
    RandomForest,
    LightGBMModel,
    XGBModel,
    RNNModel,
    TCNModel,
    NHiTSModel
)
from darts.metrics import mape, rmse, mae
from darts.dataprocessing.transformers import Scaler
import warnings
warnings.filterwarnings('ignore')

In [3]:
sales_calendar_prices = (
    pd.read_csv("../../Data/sales_train_validation.csv")
    .melt(
            id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
            var_name='d',
            value_name='sales'
    )
    .merge(pd.read_csv("../../Data/calendar.csv"), on='d', how='left')
    .merge(pd.read_csv("../../Data/sell_prices.csv"), on=['wm_yr_wk', 'item_id', 'store_id'], how='left')
)
sales_calendar_prices.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


## Utils

In [4]:
# ============================================
# Load and Prepare Data
# ============================================
def prepare_data(df, id_col='id', date_col='date', target_col='sales'):
    """
    Prepare data for Darts forecasting
    """
    # Categorical features to encode
    cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
                    'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    
    # Encode categorical variables
    df_encoded = df.copy()
    for col in cat_features:
        if col in df_encoded.columns:
            df_encoded[col] = pd.Categorical(df_encoded[col]).codes
    
    return df_encoded

def create_timeseries(df, id_value, date_col='date', target_col='sales'):
    """
    Create Darts TimeSeries for a specific item-store combination
    """
    # Filter for specific id
    df_filtered = df[df['id'] == id_value].sort_values(date_col).reset_index(drop=True)
    
    # Create target series
    target = TimeSeries.from_dataframe(
        df_filtered, 
        time_col=date_col, 
        value_cols=target_col,
        fill_missing_dates=True,
        freq='D'
    )
    
    # Create covariates (explanatory variables)
    covariate_cols = ['wday', 'month', 'year', 'item_id', 'dept_id', 'cat_id', 
                      'store_id', 'state_id', 'event_name_1', 'event_type_1', 
                      'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
    
    available_cols = [col for col in covariate_cols if col in df_filtered.columns]
    
    if available_cols:
        covariates = TimeSeries.from_dataframe(
            df_filtered,
            time_col=date_col,
            value_cols=available_cols,
            fill_missing_dates=True,
            freq='D'
        )
    else:
        covariates = None
    
    return target, covariates

In [5]:
def rmsse(train, actual, forecast):
    if hasattr(train, 'values'):
        train = train.values()
    if hasattr(actual, 'values'):
        actual = actual.values()
    if hasattr(forecast, 'values'):
        forecast = forecast.values()
    
    train = np.array(train).flatten()
    actual = np.array(actual).flatten()
    forecast = np.array(forecast).flatten()
    
    mse_forecast = np.mean((actual - forecast)**2)
    scale = np.mean(np.diff(train)**2)
    
    if scale == 0 or scale < 1e-10:
        return np.inf
    
    return np.sqrt(mse_forecast / scale)


def safe_metrics(train, actual, predicted):
    r = rmsse(train, actual, predicted)
    return {
        'RMSE': rmse(actual, predicted),  
        'MAE': mae(actual, predicted),   
        'RMSSE': r,
    }


## Models

In [6]:
# ============================================
# Configuration
# ============================================
FORECAST_HORIZON = 60  
TRAIN_TEST_SPLIT = 0.95  

print("Preparing data...")
df_prepared = prepare_data(sales_calendar_prices)
unique_ids = df_prepared['id'].unique()
del sales_calendar_prices

Preparing data...


### ARIMA, Exponential Smoothing and Prophet

In [8]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial
import numpy as np
import pandas as pd
from tqdm import tqdm

def process_single_id(id_value, df_prepared, train_test_split):
    """Process a single ID - will run in parallel"""
    try:
        target, covariates = create_timeseries(df_prepared, id_value)
        split_point = int(len(target) * train_test_split)
        train, test = target[:split_point], target[split_point:]
        cov_train, cov_test = covariates[:split_point], covariates[split_point:]
        
        results = {}
        
        # ARIMA
        model = AutoARIMA()
        model.fit(train)
        pred = model.predict(len(test))
        results['ARIMA'] = safe_metrics(train, test, pred)
        
        # Exponential Smoothing
        model = ExponentialSmoothing(seasonal_periods=7)
        model.fit(train)
        pred = model.predict(len(test))
        results['Exponential Smoothing'] = safe_metrics(train, test, pred)
        
        # Prophet
        model = Prophet()
        model.fit(train, future_covariates=cov_train)
        pred = model.predict(len(test), future_covariates=cov_test)
        results['Prophet'] = safe_metrics(train, test, pred)
        
        return id_value, results
    except Exception as e:
        print(f"Error processing {id_value}: {e}")
        return id_value, None

# Parallel execution
all_results = {}
n_workers = 6  # Adjust based on your CPU cores

np.random.seed(42)  # or any integer you prefer
sample_ids = np.random.choice(unique_ids, size=min(500, len(unique_ids)), replace=False)

with ThreadPoolExecutor(max_workers=n_workers) as executor:
    future_to_id = {
        executor.submit(process_single_id, id_val, df_prepared, TRAIN_TEST_SPLIT): id_val 
        for id_val in sample_ids
    }
    
    for future in tqdm(as_completed(future_to_id), total=len(sample_ids)):
        id_value, results = future.result()
        if results is not None:
            all_results[id_value] = results

# Rest of your code remains the same
global_results = {}
for product, models in all_results.items():
    for model_name, metrics in models.items():
        if model_name not in global_results:
            global_results[model_name] = {'RMSE': [], 'MAE': [], 'RMSSE': []}
        for metric_name, value in metrics.items():
            global_results[model_name][metric_name].append(float(value))

global_averages = {
    model_name: {metric: np.mean(values) for metric, values in metrics.items()}
    for model_name, metrics in global_results.items()
}
print(pd.DataFrame(global_averages))

  0%|          | 0/500 [00:00<?, ?it/s]16:30:44 - cmdstanpy - INFO - Chain [1] start processing
16:30:44 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 1/500 [00:08<1:11:58,  8.65s/it]16:30:49 - cmdstanpy - INFO - Chain [1] start processing
16:30:50 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 2/500 [00:13<53:35,  6.46s/it]  16:30:54 - cmdstanpy - INFO - Chain [1] start processing
16:30:54 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 3/500 [00:17<44:57,  5.43s/it]16:31:01 - cmdstanpy - INFO - Chain [1] start processing
16:31:01 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 4/500 [00:25<51:28,  6.23s/it]16:31:05 - cmdstanpy - INFO - Chain [1] start processing
16:31:06 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 5/500 [00:29<44:53,  5.44s/it]16:31:13 - cmdstanpy - INFO - Chain [1] start processing
16:31:14 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 6/500 [00:38<55:50,  6.78s/

Error processing HOUSEHOLD_1_081_TX_3_validation: 'AutoARIMA' object has no attribute '_model_call'


16:59:30 - cmdstanpy - INFO - Chain [1] start processing
16:59:30 - cmdstanpy - INFO - Chain [1] done processing
 61%|██████    | 304/500 [28:53<15:48,  4.84s/it]16:59:37 - cmdstanpy - INFO - Chain [1] start processing
16:59:37 - cmdstanpy - INFO - Chain [1] done processing
 61%|██████    | 305/500 [29:01<18:50,  5.80s/it]16:59:43 - cmdstanpy - INFO - Chain [1] start processing
16:59:43 - cmdstanpy - INFO - Chain [1] done processing
 61%|██████    | 306/500 [29:07<18:09,  5.62s/it]16:59:47 - cmdstanpy - INFO - Chain [1] start processing
16:59:47 - cmdstanpy - INFO - Chain [1] done processing
 61%|██████▏   | 307/500 [29:10<16:18,  5.07s/it]16:59:55 - cmdstanpy - INFO - Chain [1] start processing
16:59:55 - cmdstanpy - INFO - Chain [1] done processing
 62%|██████▏   | 308/500 [29:19<19:26,  6.07s/it]17:00:02 - cmdstanpy - INFO - Chain [1] start processing
17:00:02 - cmdstanpy - INFO - Chain [1] done processing
 62%|██████▏   | 309/500 [29:26<20:24,  6.41s/it]17:00:08 - cmdstanpy - INFO 

          ARIMA  Exponential Smoothing   Prophet
RMSE   1.313787               1.301794  1.386556
MAE    0.983631               0.977356  1.069421
RMSSE  0.993854               0.995023  1.018052


In [ ]:
# ============================================
# Model Training and Evaluation
# ============================================
def evaluate_models(target, covariates, forecast_horizon=60):
    """
    Train and evaluate multiple forecasting models
    """
    # Split data
    split_point = int(len(target) * TRAIN_TEST_SPLIT)
    train, test = target[:split_point], target[split_point:]
    
    if covariates is not None:
        cov_train, cov_test = covariates[:split_point], covariates[split_point:]
        # For past covariates, we need train + test
        cov_past_full = covariates[:split_point + len(test)]
    else:
        cov_train, cov_test = None, None
        cov_past_full = None
    
    results = {}
    
    # ============================================
    # 4. Random Forest (ML)
    # ============================================
    try:
        print("  Training Random Forest...")
        from darts.models import RandomForestModel  # Import the correct model
        
        model = RandomForestModel(
            lags=7,#30,
            #lags_future_covariates=[0, 1, 2, 3, 4, 5, 6, 7],
            output_chunk_length=1,
            n_estimators=100
        )
        
        # Pass future_covariates to fit and predict
        if cov_train is not None:
            model.fit(train, future_covariates=cov_train)
            pred = model.predict(len(test), future_covariates=cov_test)
        else:
            # If no covariates, create a model without them
            model = RandomForestModel(
                lags=30,
                output_chunk_length=1,
                n_estimators=100
            )
            model.fit(train)
            pred = model.predict(len(test))
        
        results['Random Forest'] = safe_metrics(test, pred, 'Random Forest')
    except Exception as e:
        print(f"  Random Forest failed: {e}")
        results['Random Forest'] = {'Error': str(e)}
    
    # ============================================
    # 5. LightGBM (ML)
    # ============================================
    try:
        print("  Training LightGBM...")
        model = LightGBMModel(
            lags=7,#30,
            #lags_future_covariates=[0, 1, 2, 3, 4, 5, 6, 7],
            output_chunk_length=1
        )
        
        # Pass future_covariates to fit and predict
        if cov_train is not None:
            model.fit(train, future_covariates=cov_train)
            pred = model.predict(len(test), future_covariates=cov_test)
        else:
            # If no covariates, create a model without them
            model = LightGBMModel(
                lags=30,
                output_chunk_length=1
            )
            model.fit(train)
            pred = model.predict(len(test))
        
        pred = scaler.inverse_transform(pred)
        results['LightGBM'] = safe_metrics(test, pred, 'LightGBM')
    except Exception as e:
        print(f"  LightGBM failed: {e}")
        results['LightGBM'] = {'Error': str(e)}
    
    # ============================================
    # 6. XGBoost (ML)
    # ============================================
    try:
        print("  Training XGBoost...")
        model = XGBModel(
            lags=7,#30,
            #lags_future_covariates=[0, 1, 2, 3, 4, 5, 6, 7],
            output_chunk_length=1
        )
        
        # Pass future_covariates to fit and predict
        if cov_train is not None:
            model.fit(train, future_covariates=cov_train)
            pred = model.predict(len(test), future_covariates=cov_test)
        else:
            # If no covariates, create a model without them
            model = XGBModel(
                lags=30,
                output_chunk_length=1
            )
            model.fit(train)
            pred = model.predict(len(test))
        
        pred = scaler.inverse_transform(pred)
        results['XGBoost'] = safe_metrics(test, pred, 'XGBoost')
    except Exception as e:
        print(f"  XGBoost failed: {e}")
        results['XGBoost'] = {'Error': str(e)}
    
    # ============================================
    # 8. TCN (Deep Learning) - only if enough data
    # ============================================
    try:
        input_chunk = min(21, len(train) // 2)
        print(f"  Training TCN (input_chunk_length={input_chunk})...")
        model = TCNModel(
            input_chunk_length=input_chunk,
            output_chunk_length=1,
            n_epochs=50,
            batch_size=32,
            random_state=42,
            pl_trainer_kwargs={"accelerator": "cpu"}  # Force CPU to avoid MPS issues
        )
        model.fit(train, verbose=False)
        pred = model.predict(len(test))
        
        results['TCN'] = safe_metrics(test, pred, 'TCN')
    except Exception as e:
        print(f"  TCN failed: {e}")
        results['TCN'] = {'Error': str(e)}
    
    # ============================================
    # 9. NHiTS (Deep Learning) - State-of-the-art
    # ============================================
    try:
        input_chunk = min(21, len(train) // 2)
        print(f"  Training NHiTS (input_chunk_length={input_chunk})...")
        model = NHiTSModel(
            input_chunk_length=input_chunk,
            output_chunk_length=1,
            n_epochs=50,
            batch_size=32,
            random_state=42,
            pl_trainer_kwargs={"accelerator": "cpu"}  # Force CPU to avoid MPS issues
        )
        model.fit(train, verbose=False)
        pred = model.predict(len(test))
        
        results['NHiTS'] = safe_metrics(test, pred, 'NHiTS')
    except Exception as e:
        print(f"  NHiTS failed: {e}")
        results['NHiTS'] = {'Error': str(e)}
    
    return results

# ============================================
# Main Execution
# ============================================
def main(df):
    """
    Main function to run forecasting experiments
    """
    print("Preparing data...")
    df_prepared = prepare_data(df)
    
    # Get unique IDs (item x store combinations)
    unique_ids = df_prepared['id'].unique()
    
    # Test on first few IDs (you can increase this)
    test_ids = unique_ids[:1]  # Test on 3 items
    
    all_results = {}
    
    for id_value in test_ids:
        print(f"\n{'='*50}")
        print(f"Processing ID: {id_value}")
        print(f"{'='*50}")
        
        try:
            target, covariates = create_timeseries(df_prepared, id_value)
            print(f"Time series length: {len(target)}")
            
            results = evaluate_models(target, covariates)
            all_results[id_value] = results
            
        except Exception as e:
            print(f"Error processing {id_value}: {e}")
            all_results[id_value] = {'Error': str(e)}
    
    # ============================================
    # Display Results
    # ============================================
    print("\n" + "="*70)
    print("RESULTS SUMMARY")
    print("="*70)
    
    for id_value, results in all_results.items():
        print(f"\n{id_value}:")
        if 'Error' in results:
            print(f"  Error: {results['Error']}")
        else:
            results_df = pd.DataFrame(results).T
            print(results_df.to_string())
    
    return all_results

# ============================================
# Usage
# ============================================
# Assuming your dataframe is called 'df'
# results = main(df)

### ML models

In [7]:
from xgboost import XGBRegressor

def forecast_with_ml(df, id_col='id', date_col='date', target_col='sales', 
                     forecast_horizon=60, train_ratio=0.8):
    """Forecast time series using XGBoost with batch processing"""
    
    # Prepare data
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([id_col, date_col]).reset_index(drop=True)
    
    # Sample IDs
    np.random.seed(42)
    sample_ids = np.random.choice(unique_ids, size=min(500, len(unique_ids)), replace=False)
    df = df[df[id_col].isin(sample_ids)]
    
    # Create train/test split indicator
    df['row_num'] = df.groupby(id_col).cumcount()
    df['total_rows'] = df.groupby(id_col)[id_col].transform('count')
    df['split_point'] = (df['total_rows'] * train_ratio).astype(int)
    df['is_train'] = df['row_num'] < df['split_point']
    df['is_test'] = (df['row_num'] >= df['split_point']) & (df['row_num'] < df['split_point'] + forecast_horizon)
    
    # Encode categorical columns
    feature_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
                    'd', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 
                    'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
    feature_cols = [c for c in feature_cols if c in df.columns]
    
    df_encoded = df.copy()
    for col in feature_cols:
        if df_encoded[col].dtype == 'object':
            df_encoded[col] = pd.Categorical(df_encoded[col]).codes
    
    # Train model on all training data (batch)
    train_mask = df['is_train']
    X_train = df_encoded.loc[train_mask, feature_cols]
    y_train = df.loc[train_mask, target_col]
    
    model = XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on all test data (batch)
    test_mask = df['is_test']
    X_test = df_encoded.loc[test_mask, feature_cols]
    df.loc[test_mask, 'pred'] = model.predict(X_test)
    
    # Calculate metrics by ID (vectorized where possible)
    results = {}
    
    for uid in df[id_col].unique():
        mask = df[id_col] == uid
        series_df = df[mask].copy()
        
        train_vals = series_df[series_df['is_train']][target_col].values
        test_vals = series_df[series_df['is_test']][target_col].values
        pred_vals = series_df[series_df['is_test']]['pred'].values
        
        if len(test_vals) > 0 and len(pred_vals) > 0:
            min_len = min(len(test_vals), len(pred_vals))
            actual = test_vals[:min_len]
            pred = pred_vals[:min_len]
            
            mae = np.mean(np.abs(actual - pred))
            rmse = np.sqrt(np.mean((actual - pred) ** 2))
            _rmsse = rmsse(train_vals, actual, pred)
        else:
            mae, rmse, _rmsse = np.nan, np.nan, np.nan
        
        results[uid] = {
            "XGB": {
                'RMSE': rmse,
                'MAE': mae,
                'RMSSE': _rmsse,
            }
        }
    
    return results

In [21]:
all_results_ml = forecast_with_ml(
     df=df_prepared,
     id_col='id',
     date_col='date', 
     target_col='sales',
     forecast_horizon=60,
     train_ratio=TRAIN_TEST_SPLIT
)

# View results
# Rest of your code remains the same
global_results = {}
for product, models in all_results_ml.items():
    for model_name, metrics in models.items():
        if model_name not in global_results:
            global_results[model_name] = {'RMSE': [], 'MAE': [], 'RMSSE': []}
        for metric_name, value in metrics.items():
            global_results[model_name][metric_name].append(float(value))

global_averages = {
    model_name: {metric: np.mean(values) for metric, values in metrics.items()}
    for model_name, metrics in global_results.items()
}
print(pd.DataFrame(global_averages))

            XGB
RMSE   1.325208
MAE    1.027778
RMSSE  1.019561


In [17]:
from darts import TimeSeries
from darts.models import XGBModel
from darts.dataprocessing.transformers import Scaler


def forecast_with_ml_lags(df, id_col='id', date_col='date', target_col='sales', 
                          forecast_horizon=60, train_ratio=0.8, lags=None):
    """
    Forecast time series using XGBoost via Darts with lag features
    Trains a SINGLE model on ALL series together
    
    Parameters:
    -----------
    lags : list or int, optional
        Lag values to use. If int, uses lags from 1 to that value.
        Default: [-1, -7, -14, -28] (good for daily retail data)
    """
    
    if lags is None:
        lags = [-1, -7, -14, -28]
    elif isinstance(lags, int):
        lags = [-i for i in range(1, lags + 1)]
    
    # Prepare data
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([id_col, date_col]).reset_index(drop=True)
    
    # Sample IDs
    np.random.seed(42)
    sample_ids = np.random.choice(unique_ids, size=min(500, len(unique_ids)), replace=False)
    df_sample = df[df[id_col].isin(sample_ids)]
    
    # Prepare future covariates
    covariate_cols = ['wday', 'month', 'year', 'event_name_1', 'event_type_1', 
                      'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
    covariate_cols = [c for c in covariate_cols if c in df_sample.columns]
    
    # Encode categorical columns
    df_encoded = df_sample.copy()
    for col in covariate_cols:
        if df_encoded[col].dtype == 'object':
            df_encoded[col] = pd.Categorical(df_encoded[col]).codes
    
    # Group by ID and prepare all series
    grouped = df_encoded.groupby(id_col)
    
    all_train_series = []
    all_train_cov = []
    all_test_data = []
    max_lag = abs(min(lags))
    
    # Prepare all series for batch training
    for uid, series_df in grouped:
        if len(series_df) < max_lag + forecast_horizon:
            continue
        
        # Create TimeSeries
        target_series = TimeSeries.from_dataframe(
            series_df, time_col=date_col, value_cols=target_col,
            fill_missing_dates=True, freq='D'
        )
        
        future_covariates = None
        if covariate_cols:
            future_covariates = TimeSeries.from_dataframe(
                series_df, time_col=date_col, value_cols=covariate_cols,
                fill_missing_dates=True, freq='D'
            )
        
        # Split train/test
        split_point = int(len(target_series) * train_ratio)
        train_series = target_series[:split_point]
        test_series = target_series[split_point:split_point + forecast_horizon]
        
        if len(test_series) == 0:
            continue
        
        # Collect training data
        all_train_series.append(train_series)
        
        if future_covariates is not None:
            train_cov = future_covariates[:split_point]
            future_cov = future_covariates[:split_point + forecast_horizon]
            all_train_cov.append(train_cov)
        
        # Store test info for later
        all_test_data.append({
            'uid': uid,
            'train_vals': train_series.values().flatten(),
            'test_vals': test_series.values().flatten(),
            'test_series': test_series,
            'train_series': train_series,  # Need this for prediction
            'future_cov': future_cov if future_covariates is not None else None
        })
    
    # Train ONE model on ALL series
    model = XGBModel(
        lags=lags,
        lags_future_covariates=[0] if covariate_cols else None,
        output_chunk_length=forecast_horizon,
        random_state=42
    )
    
    model.fit(
        series=all_train_series,
        future_covariates=all_train_cov if all_train_cov else None
    )
    
    # Predict for all series and calculate metrics
    results = {}
    
    for test_info in all_test_data:
        try:
            # Predict for this series - need to pass the training series so model can extract lags
            prediction = model.predict(
                n=len(test_info['test_series']),
                series=test_info['train_series'],  # Pass training series, not test
                future_covariates=test_info['future_cov']
            )
            
            pred_vals = prediction.values().flatten()
            test_vals = test_info['test_vals']
            train_vals = test_info['train_vals']
            
            # Calculate metrics
            min_len = min(len(test_vals), len(pred_vals))
            actual = test_vals[:min_len]
            pred = pred_vals[:min_len]
            
            mae = np.mean(np.abs(actual - pred))
            rmse = np.sqrt(np.mean((actual - pred) ** 2))
            _rmsse = rmsse(train_vals, actual, pred)
            
        except Exception as e:
            print(f"Error predicting ID {test_info['uid']}: {str(e)}")
            mae, rmse, _rmsse = np.nan, np.nan, np.nan
        
        results[test_info['uid']] = {
            "XGB_Darts": {
                'RMSE': rmse,
                'MAE': mae,
                'RMSSE': _rmsse,
            }
        }
    
    return results


In [18]:
all_results_ml_lags = forecast_with_ml_lags(
     df=df_prepared,
     id_col='id',
     date_col='date', 
     target_col='sales',
     forecast_horizon=60,
     train_ratio=TRAIN_TEST_SPLIT
)

# View results
# Rest of your code remains the same
global_results = {}
for product, models in all_results_ml_lags.items():
    for model_name, metrics in models.items():
        if model_name not in global_results:
            global_results[model_name] = {'RMSE': [], 'MAE': [], 'RMSSE': []}
        for metric_name, value in metrics.items():
            global_results[model_name][metric_name].append(float(value))

global_averages = {
    model_name: {metric: np.mean(values) for metric, values in metrics.items()}
    for model_name, metrics in global_results.items()
}
print(pd.DataFrame(global_averages))

       XGB_Darts
RMSE    1.360555
MAE     1.067951
RMSSE   1.028694


In [13]:
from typing import List, Optional, Union
def forecast_with_ml_lags_price(
    df: pd.DataFrame,
    id_col: str = "id",
    date_col: str = "date",
    target_col: str = "sales",
    forecast_horizon: int = 60,
    train_ratio: float = 0.8,
    lags: Optional[Union[List[int], int]] = None,
    past_cov_cols: Optional[List[str]] = None,
    future_cov_cols: Optional[List[str]] = None,
    sample_size: int = 500,
    random_state: int = 42,
):
    """
    Train a single XGBoost model (via Darts) on multiple series using lag features.
    Returns a DataFrame of per-series metrics (RMSE, MAE, RMSSE).

    Simplified & robust: consistent categorical encoding, fixes undefined vars, clearer splits.
    """

    # defaults
    if lags is None:
        lags = [-1, -7, -14, -28]
    elif isinstance(lags, int):
        lags = [-i for i in range(1, lags + 1)]

    if future_cov_cols is None:
        future_cov_cols = [
            "wday", "month", "year", "event_name_1", "event_type_1",
            "event_name_2", "event_type_2", "snap_CA", "snap_TX", "snap_WI",
        ]
    if past_cov_cols is None:
        past_cov_cols = []

    # prepare dataframe
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([id_col, date_col]).reset_index(drop=True)

    # consistent categorical encoding across the whole dataframe
    cov_cols = [c for c in (future_cov_cols + past_cov_cols) if c in df.columns]
    for c in cov_cols:
        if df[c].dtype == "object" or pd.api.types.is_categorical_dtype(df[c]):
            df[c] = pd.Categorical(df[c]).codes.astype(float)

    # sample ids (stable)
    np.random.seed(random_state)
    sample_ids = np.random.choice(unique_ids, size=min(sample_size, len(unique_ids)), replace=False)
    df_sample = df[df[id_col].isin(sample_ids)]

    grouped = df_sample.groupby(id_col)

    max_lag = abs(min(lags))
    train_series_list = []
    train_future_cov_list = []   # future covariates covering the training window
    train_past_cov_list = []     # past covariates covering the training window
    tests = []                   # list of dicts with test info per series

    for uid, g in grouped:
        g = g.sort_values(date_col)
        if len(g) < max_lag + 2 + forecast_horizon:  # require a little extra history
            continue

        # build TimeSeries objects
        target_ts = TimeSeries.from_dataframe(g, time_col=date_col, value_cols=target_col, fill_missing_dates=True, freq="D")

        fut_cov_ts = None
        if any(col in g.columns for col in future_cov_cols):
            present_fut = [c for c in future_cov_cols if c in g.columns]
            fut_cov_ts = TimeSeries.from_dataframe(g, time_col=date_col, value_cols=present_fut, fill_missing_dates=True, freq="D")

        past_cov_ts = None
        if any(col in g.columns for col in past_cov_cols):
            present_past = [c for c in past_cov_cols if c in g.columns]
            past_cov_ts = TimeSeries.from_dataframe(g, time_col=date_col, value_cols=present_past, fill_missing_dates=True, freq="D")

        # split: keep original ratio but ensure we have forecast_horizon in the test slice
        split_point = int(len(target_ts) * train_ratio)
        # ensure there is room for forecast horizon
        if split_point + forecast_horizon > len(target_ts):
            split_point = len(target_ts) - forecast_horizon
        if split_point <= max_lag:
            continue

        train_ts = target_ts[:split_point]
        test_ts = target_ts[split_point: split_point + forecast_horizon]
        if len(test_ts) == 0:
            continue

        train_series_list.append(train_ts)
        # training covariates must align with the training window
        train_future_cov_list.append(fut_cov_ts[:split_point] if fut_cov_ts is not None else None)
        train_past_cov_list.append(past_cov_ts[:split_point] if past_cov_ts is not None else None)

        # Store test info
        full_future_for_pred = fut_cov_ts[:split_point + len(test_ts)] if fut_cov_ts is not None else None
        full_past_for_pred = past_cov_ts[:split_point] if past_cov_ts is not None else None

        tests.append({
            "uid": uid,
            "train_series": train_ts,
            "train_vals": train_ts.values().flatten(),
            "test_series": test_ts,
            "test_vals": test_ts.values().flatten(),
            "future_cov_full": full_future_for_pred,
            "past_cov_full": full_past_for_pred,   # <-- NEW
        })

    if len(train_series_list) == 0:
        raise ValueError("No series qualified for training (increase sample_size or reduce forecast_horizon).")

    # decide which covariates to pass to fit()
    future_cov_for_fit = train_future_cov_list if any(x is not None for x in train_future_cov_list) else None
    past_cov_for_fit = train_past_cov_list if any(x is not None for x in train_past_cov_list) else None

    print("fit")
    # create and fit the model
    model = XGBModel(
        lags=lags,
        lags_future_covariates=[0] if future_cov_for_fit is not None else None, 
        lags_past_covariates=[-1],
        output_chunk_length=forecast_horizon,
        random_state=random_state,
    )

    model.fit(series=train_series_list,
              future_covariates=future_cov_for_fit,
              past_covariates=past_cov_for_fit)

    # predict & evaluate
    records = {}
    for t in tests:
        try:
            pred_ts = model.predict(
                n=len(t["test_series"]),
                series=t["train_series"],
                future_covariates=t["future_cov_full"],
                past_covariates=t["past_cov_full"],   # <-- NEW
            )
            pred_vals = pred_ts.values().flatten()
            actual = t["test_vals"]
            train_vals = t["train_vals"]

            min_len = min(len(actual), len(pred_vals))
            actual = actual[:min_len]
            pred_vals = pred_vals[:min_len]

            mae = float(np.mean(np.abs(actual - pred_vals)))
            rmse = float(np.sqrt(np.mean((actual - pred_vals) ** 2)))
            rmsse_val = rmsse(train_vals, actual, pred_vals)
        except Exception as e:
            mae = rmse = rmsse_val = np.nan

        uid = t["uid"]
        records[uid] = {
            "XGB_Darts": {
                'RMSE': rmse,
                'MAE': mae,
                'RMSSE': rmsse_val,
            }
        }

    return records

In [14]:
all_results_ml_lags_price = forecast_with_ml_lags_price(
    df_prepared, 
    sample_size=500,
    train_ratio=TRAIN_TEST_SPLIT,
    past_cov_cols=['sell_price'],  # Won't have these in future
    future_cov_cols=['wday', 'month', 'year', 'event_name_1', 'event_type_1', 
                      'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']  # Will have these
)

# View results
# Rest of your code remains the same
global_results = {}
for product, models in all_results_ml_lags_price.items():
    for model_name, metrics in models.items():
        if model_name not in global_results:
            global_results[model_name] = {'RMSE': [], 'MAE': [], 'RMSSE': []}
        for metric_name, value in metrics.items():
            global_results[model_name][metric_name].append(float(value))

global_averages = {
    model_name: {metric: np.mean(values) for metric, values in metrics.items()}
    for model_name, metrics in global_results.items()
}
print(pd.DataFrame(global_averages))

fit
       XGB_Darts
RMSE    1.360145
MAE     1.057615
RMSSE   1.034130


### Foundational Model Google

In [9]:
import timesfm
import torch
from tqdm import tqdm

def initialize_model():
    """Initialize TimesFM model (call once)"""
    torch.set_float32_matmul_precision("high")
    
    model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")
    
    model.compile(
        timesfm.ForecastConfig(
            max_context=1024,
            max_horizon=256,
            normalize_inputs=True,
            use_continuous_quantile_head=True,
            force_flip_invariance=True,
            infer_is_positive=True,
            fix_quantile_crossing=True,
        )
    )
    
    return model
    
# ============================================
# Batch Version (More Efficient)
# ============================================
def forecast_with_timesfm_batch(df, id_col='id', date_col='date', target_col='sales', 
                                forecast_horizon=60, train_ratio=0.8):
    """
    Batch version for faster processing of multiple time series
    """
    
    # 1. Load TimesFM model
    print("Loading TimesFM model...")
    model = initialize_model()
    
    # 2. Prepare data
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([id_col, date_col])
        
    # 3. Collect all series
    all_train = []
    all_test = []
    valid_ids = []
    
    np.random.seed(42)  # or any integer you prefer
    sample_ids = np.random.choice(unique_ids, size=min(500, len(unique_ids)), replace=False)
    grouped = df[df[id_col].isin(sample_ids)].groupby(id_col)[target_col].apply(lambda x: x.values)
    
    for uid, series_data in grouped.items():
        split = int(len(series_data) * train_ratio)
        train = series_data[:split]
        test = series_data[split:split + forecast_horizon]
        
        all_train.append(train)
        all_test.append(test)
        valid_ids.append(uid)
    
    # 4. Batch forecast (all at once!)
    print(f"Forecasting {len(valid_ids)} series in one batch...")
    point_forecast, quantile_forecast = model.forecast(
        horizon=forecast_horizon,
        inputs=all_train,  # All time series at once
    )
    
    # 5. Process results
    results = {}
    for i, uid in enumerate(tqdm(valid_ids)):
        predictions = point_forecast[i]
        test = all_test[i]
        train = all_train[i]
        
        # Calculate metrics
        if len(test) > 0:
            min_len = min(len(test), len(predictions))
            actual = test[:min_len]
            pred = predictions[:min_len]
            train = all_train[i]
            
            mae = np.mean(np.abs(actual - pred))
            rmse = np.sqrt(np.mean((actual - pred) ** 2))
            _rmsse = rmsse(train, test, pred)
        else:
            mae, rmse, _rmsse = None, None, None
        
        results[uid] = {"FM": {
            'RMSE': rmse,  
            'MAE': mae,   
            'RMSSE': _rmsse,
        }}
    
    return results

In [11]:
all_results_fm = forecast_with_timesfm_batch(
     df=df_prepared,
     id_col='id',
     date_col='date', 
     target_col='sales',
     forecast_horizon=60,
     train_ratio=TRAIN_TEST_SPLIT
)

# View results
# Rest of your code remains the same
global_results = {}
for product, models in all_results_fm.items():
    for model_name, metrics in models.items():
        if model_name not in global_results:
            global_results[model_name] = {'RMSE': [], 'MAE': [], 'RMSSE': []}
        for metric_name, value in metrics.items():
            global_results[model_name][metric_name].append(float(value))

global_averages = {
    model_name: {metric: np.mean(values) for metric, values in metrics.items()}
    for model_name, metrics in global_results.items()
}
print(pd.DataFrame(global_averages))

Loading TimesFM model...


config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

Downloaded.
Forecasting 500 series in one batch...


100%|██████████| 500/500 [00:00<00:00, 22113.93it/s]


             FM
RMSE   1.276774
MAE    0.833277
RMSSE  0.979278
